In [140]:
import os
import pandas as pd
import numpy as np
import math

In [141]:
def IssuerCode(state=False):
    issuers = pd.read_csv('data/s&p500.csv',sep=';')
    issuer_filtered = []
    for code_stock in issuers.Ticker:
        # Check last transaction 2023
        # Have 200 transaction
        path = f'data/transactions/{code_stock}.csv'
        if os.path.exists(path):
            historical = pd.read_csv(path)
            last_date_row = historical.Date.iloc[len(historical)-1] # get last date transactions
            data_test = historical[-1000:]['Close'] # get data test
            var = np.var(data_test) # calculate variance from data test
            if (last_date_row[:4] == '2023' and len(historical) >= 200 and var > 0):
                issuer_filtered.append(code_stock)
    if state:
        print("Code of Stock Issuer: {}".format(issuer_filtered))
    else:
        return issuer_filtered

In [142]:
issuers = IssuerCode()

In [143]:
def get_signal(df_pred):
    arr_signal = []
    lot = 0
    for k in range(len(df_pred)):
        action = "HOLD"
        if k > 0:
            prev = df_pred['y_predict'][k-1]
            current = df_pred['y_predict'][k]
            if current < prev and lot == 0:
                action = "BUY"
                lot = current
            elif current > lot and lot != 0:
                lot = 0
                action = "SELL"
        arr_signal.append(action)
    return arr_signal

In [144]:
def trading(signals, df_test, cash=1000):
    lot = 0
    cash_beli = 0
    #harga dari dataset = harga per lembar
    for k in range(len(signals)):
        action = signals[k]
        open_price = df_test['Open'].iloc[k]
        close_price = df_test['Close'].iloc[k]
        if action == "BUY" and lot == 0 and cash > (open_price*100):
            lot += math.floor(cash/(open_price*100))
            cash_beli = open_price * (lot*100)
            cash -= cash_beli
        elif action == "SELL" and lot > 0:
            cash += close_price * (lot*100)
            lot = 0

        #last rows
        #1000 dan masih hold, dijual
        if k == (len(signals)-1) and lot > 0:
            cash += cash_beli
    return cash

In [145]:
# lookbacks = [5,10,15,20,25]
# architecture = ['TFGRU', 'TFLSTM', 'TFGRULSTM', 'TFLSTMGRU']
lookbacks = [20]
architecture = ['TFGRU']

issuer_result = {'code':[],'lookback':[],'arch':[],'cash_forecast':[], 'cash_sr':[]}

path_result = 'results/targets/'
initial_cash = 1000000
for code in issuers:
    path = f'data/transactions/{code}.csv'
    if os.path.exists(path):
        df = pd.read_csv(path)
        df_test = df[-1000:]
        
        for archi in architecture:
            for lookback in lookbacks:
                path_target = f'{path_result}{archi}_multivariate/{code}_{lookback}_test_dfy.csv'
                df_pred = pd.read_csv(path_target)
                
                arr_signal = get_signal(df_pred)
                cash = trading(arr_signal, df_test, initial_cash)
                issuer_result['code'].append(code)
                issuer_result['lookback'].append(lookback)
                issuer_result['arch'].append(archi)
                issuer_result['cash_forecast'].append(cash)        


In [146]:
for code in issuers:
    path = f'data/transactions/{code}.csv'
    if os.path.exists(path):
        df = pd.read_csv(path)
        df_test = df[-1000:]
        path_signal = f'results/support_resistance_signals/{code}_signals.csv'
        
        arr_signal = pd.read_csv(path_signal)
        cash = trading(arr_signal['action'], df_test, initial_cash)
        issuer_result['cash_sr'].append(cash) 

In [147]:
# all_result
df_result = pd.DataFrame(issuer_result)
df_result['return_forecast'] = df_result['cash_forecast'] - initial_cash
df_result['return_sr'] = df_result['cash_sr'] - initial_cash
df_result['return_forecast_percent'] = df_result['return_forecast'] / initial_cash * 100
df_result['return_sr_percent'] = df_result['return_sr'] / initial_cash * 100
#diatas 0% sudah untung
print(df_result)

    code  lookback   arch  cash_forecast       cash_sr  return_forecast  \
0   AAPL        20  TFGRU   3.567230e+06  1.144020e+06     2.567230e+06   
1   MSFT        20  TFGRU   1.993725e+06  1.231722e+06     9.937252e+05   
2   AMZN        20  TFGRU   1.851691e+06  7.970617e+05     8.516907e+05   
3   NVDA        20  TFGRU   5.718152e+06  1.781520e+06     4.718152e+06   
4  GOOGL        20  TFGRU   2.929150e+06  6.267817e+05     1.929150e+06   
5   TSLA        20  TFGRU   1.058960e+07  4.912080e+05     9.589596e+06   
6   META        20  TFGRU   1.922054e+06  2.343085e+06     9.220539e+05   
7   GOOG        20  TFGRU   2.870630e+06  7.432552e+05     1.870630e+06   
8    UNH        20  TFGRU   2.446303e+06  1.004398e+06     1.446303e+06   
9    XOM        20  TFGRU   1.744140e+06  9.732857e+05     7.441400e+05   

      return_sr  return_forecast_percent  return_sr_percent  
0  1.440200e+05               256.722993          14.402001  
1  2.317219e+05                99.372518          

In [148]:
# sum_all_code_result
df_result = pd.DataFrame(issuer_result)
df_sums = df_result.groupby(['arch', 'lookback'], as_index=False).sum()[['arch', 'lookback', 'cash_forecast', 'cash_sr']]
print(df_sums)

    arch  lookback  cash_forecast       cash_sr
0  TFGRU        20   3.563267e+07  1.113634e+07
